## Описание проекта
Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час.

## Цель
Построить модель для предсказания количества таки в аэропортах на следующий час.

## Требования
Значение метрики RMSE на тестовой выборке должно быть не больше 48.

## Описание данных
Количество заказов находится в столбце 'num_orders' (от англ. number of orders, «число заказов»).

Импортируем все нужное

In [131]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error

from catboost import CatBoostRegressor

import plotly.express as px

from scipy.stats import shapiro

from statsmodels.tsa.seasonal import seasonal_decompose

## Шаг 1. Загрузим и посмотрим на данные.

In [20]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/taxi.csv', index_col=[0], parse_dates=[0])
data.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


Проверим данные на монотонность

In [22]:
if data.index.is_monotonic_increasing:
    print('Монотонный временной ряд, данные распределены от прошлого к будущему.')
else:
    print('Данные в датасете перемешаны во времени.')

Монотонный временной ряд, данные распределены от прошлого к будущему.


Проверим, что в данных нет одинаковых моментов времени

In [23]:
if data.index.is_unique:
    print('Временные метки уникальны')
else:
    print('Временные метки не уникальные')

Временные метки уникальны


Проверим данные на пропуски

In [24]:
print(f'Количество пропусков в данных: {data.isna().sum()[0]}')

Количество пропусков в данных: 0


Ресемплируем данные по часу.

In [25]:
data_by_hour = data.resample('1h').sum()

Построим гистограмму количества заказов

In [26]:
fig = px.histogram(
    data_by_hour,
    title='Распределение количества заказов'
)

fig.update_xaxes(
    title='Количество заказов '
)

fig.update_yaxes(
    title='Количество'
)

fig.update_layout(
    showlegend=False,
    bargap=0.1,
)

fig.show()

Гистограмма напоминает нормальное распределение, но присутствует смещение вправо, а также иногда встречается аномально большое количество заказов.

In [74]:
decompose = seasonal_decompose(data_by_hour)

In [39]:
fig = px.scatter(
    decompose.trend,
    title='Тренд количества заказов такси'
)

fig.update_xaxes(
    title='Дата'
)

fig.update_yaxes(
    title='Количество заказов такси'
)

fig.update_layout(
    showlegend=False
)

fig.show()

Наблюдается тренд увеличения количества заказов такси. Также видим падение количества заказов по ночам.

Посмотрим есть ли какая-то сезонность.

In [65]:
fig = px.scatter(
    decompose.seasonal['2018-08-01':'2018-08-01'],
    title='Тренд количества заказов такси'
)

fig.update_xaxes(
    title='Дата'
)

fig.update_yaxes(
    title='Количество заказов такси'
)

fig.update_layout(
    showlegend=False
)

fig.show()

Сезонность в данных наблюдается только в течение дня. С 5 до 7 утра количество заказов минимально. Наблюдается стабильный рост заказов с 18 часов и до полуночи. Максимум заказов наблюдается в районе полуночи.

## Шаг 2. Создание признаков

Напишем функцию создания признаков.

In [76]:
def make_features(data: pd.DataFrame, max_lag: int, rolling_mean_size: int) -> pd.DataFrame:
    # создадим признаки с годом, месяцем, днем и днем недели
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek

    # создадим признаки со смещение по времени
    for lag in range(1, max_lag + 1):
        data[f'lag_{lag}'] = data['num_orders'].shift(lag)

    # признак со скользящим средним
    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()
    # признак со скользящим стандартным отклонением
    data['rolling_std'] = data['num_orders'].shift().rolling(rolling_mean_size).std()

    return data.dropna()

Создадим признаки

In [77]:
data_by_hour_with_features = make_features(data_by_hour, 200, 200).dropna()

C:\Users\HardWorkingStation\AppData\Local\Temp\ipykernel_23684\786761204.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\HardWorkingStation\AppData\Local\Temp\ipykernel_23684\786761204.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\HardWorkingStation\AppData\Local\Temp\ipykernel_23684\786761204.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

## Шаг 3. Обучение модели.

Разделим данные на обучающую и тестовую выборки

In [95]:
train, test = train_test_split(data_by_hour_with_features, test_size=.25, shuffle=False)

train_features = train.drop('num_orders', axis=1)
train_target = train['num_orders']

test_features = test.drop('num_orders', axis=1)
test_target = test['num_orders']

Обучим модель Случайного леса

In [111]:
rfr_params = dict(
    n_estimators = range(50, 300, 10),
    max_depth = range(3, 10, 1),
    min_samples_split = range(2, 10, 1),
    min_samples_leaf = range(2, 3, 1)
)

rfr = RandomForestRegressor(random_state=25, n_jobs=-1)
random_search_cv = RandomizedSearchCV(rfr, rfr_params, cv=3, n_iter=15, n_jobs=-1, scoring='neg_root_mean_squared_error')
rfr = random_search_cv.fit(train_features, train_target)

In [113]:
print(f'RMSE полученного RandomForestRegressor: {abs(rfr.best_score_):.2f}')
print('Параметры RandomForestRegressor')
print(rfr.best_params_)
print(f'RMSE на тестовой выборке: {mean_squared_error(test_target, rfr.predict(test_features), squared=False)}')

RMSE RandomForestRegressor: 20.29
Параметры RandomForestRegressor
{'n_estimators': 130, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_depth': 9}


Обучим модель линейной регрессии

In [115]:
lr = LinearRegression()
lr.fit(train_features, train_target)
test_pred = lr.predict(test_features)
print(f'RMSE на тестовой выборке: {mean_squared_error(test_target, test_pred, squared=False)}')

RMSE: 34.012380058552644


Обучим модель Ridge

In [124]:
ridge = Ridge(random_state=25)

ridge_params = dict(
    alpha = np.arange(0.1, 1.0, 0.1)
)

random_search_cv = GridSearchCV(ridge, ridge_params, cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error')
ridge_cv = random_search_cv.fit(train_features, train_target)

In [127]:
print(f'RMSE полученной модели Ridge: {abs(ridge_cv.best_score_):.2f}')
print('Параметры Ridge')
print(ridge_cv.best_params_)
print(f'RMSE на тестовой выборке: {mean_squared_error(test_target, ridge_cv.predict(test_features), squared=False)}')

RMSE полученной модели Ridge: 20.16
Параметры Ridge
{'alpha': 0.1}
RMSE на тестовой выборке: 34.01145429991619


Обучим модель HistGradientBoostingRegressor

In [129]:
hist_boost = HistGradientBoostingRegressor(scoring='neg_root_mean_squared_error', random_state=25)

hist_boost_params = dict(
    learning_rate = np.arange(0.1, 1.0, 0.1),
    max_depth = range(2, 10, 1),
    l2_regularization = np.arange(0.1, 1.0, 0.1),

)

hist_boost_cv = RandomizedSearchCV(hist_boost, hist_boost_params, n_iter=15 ,cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error')
hist_boost_cv = hist_boost_cv.fit(train_features, train_target)

In [130]:
print(f'RMSE полученной модели HistGradientBoostingRegressor: {abs(hist_boost_cv.best_score_):.2f}')
print('Параметры HistGradientBoostingRegressor')
print(hist_boost_cv.best_params_)
print(f'RMSE на тестовой выборке: {mean_squared_error(test_target, hist_boost_cv.predict(test_features), squared=False)}')

RMSE полученной модели HistGradientBoostingRegressor: 20.07
Параметры HistGradientBoostingRegressor
{'max_depth': 4, 'learning_rate': 0.1, 'l2_regularization': 0.9}
RMSE на тестовой выборке: 37.52521595398865


Обучим модель CatBoostRegressor

In [132]:
cbr = CatBoostRegressor(loss_function='RMSE', random_seed=25, verbose=False)

cbr_params = dict(
    max_depth = range(2, 10, 1),
    n_estimators = range(500, 1500, 100),
    learning_rate = np.arange(0.1, 1.0, 0.1),
    min_data_in_leaf = range(2, 10, 1)

)

cbr_cv = cbr.randomized_search(cbr_params, train_features, train_target, cv=3, n_iter=15, shuffle=False, train_size=0.9, verbose=False)


bestTest = 23.78388811
bestIteration = 87


bestTest = 27.62351771
bestIteration = 11


bestTest = 24.63675422
bestIteration = 9


bestTest = 23.80782476
bestIteration = 19


bestTest = 27.37897078
bestIteration = 20


bestTest = 28.76922518
bestIteration = 15


bestTest = 25.3909197
bestIteration = 13


bestTest = 27.37897078
bestIteration = 20


bestTest = 31.11139947
bestIteration = 14


bestTest = 26.4457682
bestIteration = 9


bestTest = 25.35172546
bestIteration = 29


bestTest = 25.27350073
bestIteration = 33


bestTest = 25.212745
bestIteration = 118


bestTest = 29.83634716
bestIteration = 7


bestTest = 23.75527745
bestIteration = 330

Training on fold [0/3]

bestTest = 18.10523778
bestIteration = 68

Training on fold [1/3]

bestTest = 20.92621381
bestIteration = 114

Training on fold [2/3]

bestTest = 22.3313645
bestIteration = 35



In [154]:
print(f"RMSE полученной модели HistGradientBoostingRegressor: {cbr.best_score_['learn']['RMSE']:.2f}")
print('Параметры HistGradientBoostingRegressor')
print(cbr.get_params())
print(f'RMSE на тестовой выборке: {mean_squared_error(test_target, cbr.predict(test_features), squared=False)}')

RMSE полученной модели HistGradientBoostingRegressor: 5.94
Параметры HistGradientBoostingRegressor
{'loss_function': 'RMSE', 'random_seed': 25, 'verbose': False, 'min_data_in_leaf': 9, 'depth': 3, 'iterations': 1000, 'learning_rate': 0.2}
RMSE на тестовой выборке: 38.74359001457287
